In [1]:
#import sys
#print(sys.version)

In [2]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import ultralytics
ultralytics.__version__

/Users/carlotacortes/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'8.3.7'

In [4]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [5]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [6]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [7]:
#llegir el video
cap=cv2.VideoCapture('/Users/carlotacortes/Desktop/video1min.mp4')

In [8]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]

In [9]:
#llegint tots els frames del video fins que no hi hagin + frames a llegir
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    #resize els frames a mida 1020x500
    frame=cv2.resize(frame,(1020,500))
   
    #fent servir YOLO per fer prediction en cada frame. Detection a cada frame.
    results=model.predict(frame)
 #   print(results)
    #aconseguint les dades de la bounding box i confiança
    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  # ???
    #crear aquest dataframe així tenim cada valor en cada columna:
    #   bounding box es: Xmin, Ymin, Xmax, Ymax
    #   confidence
    #   class id
    px=pd.DataFrame(a).astype("float")
    #print(px)

    #only work with car class. only count cars. Quedarnos bounding box dels cars
    list=[]    
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            #bounding box coordinates
            list.append([x1,y1,x2,y2])
            #print(c)


    #provide les coordinades al tracker. 
    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2


        ###ACTUALITZAR AMB ELS NOSTRES Y !!!!
        red_line_y=400
        blue_line_y=450   
        offset = 7
        

        # condicio per cotxes que fan ordre: linea vermella i després blava. ENTRANT. DOWNWARDS.
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
          down[id]=cy   
        if id in down:
           if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):  
             #dibuixar punts centre bounding box               
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
             #counter+=1
             counter_down.append(id)  #llista

        # condicio per cotxes que fan ordre: linea blava i després vermella. SORTINT. UPWARDS. 
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
          up[id]=cy   
        if id in up:
           if red_line_y < (cy + offset) and red_line_y > (cy - offset):
             #dibuixar punts centre bounding box        
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
             #counter+=1
             counter_up.append(id)  #llista 


    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    blue_color = (255, 0, 0)  # (B, G, R)
    #green_color = (0, 255, 0)  # (B, G, R)  

    #dibuix LINEA VERMELLA
    cv2.line(frame,(170,400),(800,400),red_color,3)
    
    #dibuix LINEA BLAVA
    cv2.line(frame,(165,450),(850,450),blue_color,3)  


    downwards = (len(counter_down))
    print("baixant: ", downwards)
    
    upwards = (len(counter_up))
    print("pujant: ", upwards)
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

/Users/carlotacortes/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 320x640 3 cars, 1 bus, 423.0ms
Speed: 4.6ms preprocess, 423.0ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 640)
baixant:  0
pujant:  0

0: 320x640 3 cars, 392.1ms
Speed: 3.1ms preprocess, 392.1ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)
baixant:  0
pujant:  0

0: 320x640 3 cars, 353.3ms
Speed: 2.0ms preprocess, 353.3ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)
baixant:  0
pujant:  0

0: 320x640 3 cars, 362.1ms
Speed: 2.2ms preprocess, 362.1ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)
baixant:  0
pujant:  0

0: 320x640 3 cars, 369.9ms
Speed: 2.5ms preprocess, 369.9ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)
baixant:  0
pujant:  0

0: 320x640 3 cars, 335.4ms
Speed: 2.8ms preprocess, 335.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
baixant:  0
pujant:  0

0: 320x640 3 cars, 398.4ms
Speed: 2.1ms preprocess, 398.4ms inference, 1.2ms postprocess per image 

KeyboardInterrupt: 